# Arbeitspaket (AP) 3: Management & Nutzung Räumliche Daten

### Angaben Studierende(r) (fehlende Angaben ergänzen)

<table>
  <tr>
    <td>Vorname:</td>
    <td>Andrin</td>
  </tr>
  <tr>
    <td>Nachname:</td>
    <td>Gloor</td>
  </tr>
  <tr>
    <td>Immatrikulationsnummer:</td>
    <td>20-649-810</td>
  </tr>
  <tr>
    <td>Modul:</td>
    <td>Data Science</td>
  </tr>
  <tr>
    <td>Prüfungsdatum / Raum / Zeit:</td>
    <td>06.10.2025 / Raum: MU O2.001 / 8:00 – 11:45</td>
  </tr>
  <tr>
    <td>Erlaubte Hilfsmittel:</td>
    <td>w.MA.XX.DS.25HS (Data Science)<br>Open Book, Eigener Computer, Internet-Zugang</td>
  </tr>
  <tr>
  <td>Nicht erlaubt:</td>
  <td>Nicht erlaubt ist der Einsatz beliebiger Formen von generativer KI (z.B. Copilot, ChatGPT) <br> sowie beliebige Formen von Kommunikation oder Kollaboration mit anderen Menschen.</td>
</tr>
</table>

## Bewertungskriterien

### <b style="color: gray;">(max. erreichbare Punkte: 40)</b>

<table>
  <thead>
    <tr>
      <th>Kategorie</th>
      <th>Beschreibung</th>
      <th>Punkteverteilung</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>Code nicht lauffähig oder Ergebnisse nicht sinnvoll</td>
      <td>Der Code enthält Fehler, die verhindern, dass er ausgeführt werden kann (z.B. Syntaxfehler) oder es werden Ergebnisse ausgegeben, welche nicht zur Fragestellung passen.</td>
      <td>0 Punkte</td>
    </tr>
    <tr>
      <td>Code lauffähig, aber mit gravierenden Mängeln</td>
      <td>Der Code läuft, aber die Ergebnisse sind aufgrund wesentlicher Fehler unvollständig (z.B. fehlende Joins, gravierende Fehler in SQL-Abfragen). Nur geringer Fortschritt erkennbar.</td>
      <td>25% der max. erreichbaren Punkte</td>
    </tr>
    <tr>
      <td>Code lauffähig, aber mit mittleren Mängeln</td>
      <td>Der Code läuft und liefert teilweise korrekte Ergebnisse, aber es gibt grössere Fehler (z.B. fehlende Spalten, unvollständige SQL-Abfragen). Die Ergebnisse sind nachvollziehbar, aber unvollständig oder ungenau.</td>
      <td>50% der max. erreichbaren Punkte</td>
    </tr>
    <tr>
      <td>Code lauffähig, aber mit minimalen Mängeln</td>
      <td>Der Code läuft und liefert ein weitgehend korrektes Ergebnis, aber kleinere Fehler (z.B. falsche oder fehlende Sortierung, Rundung von Werten falsch) beeinträchtigen die Vollständigkeit des Ergebnisses.</td>
      <td>75% der max. erreichbaren Punkte</td>
    </tr>
    <tr>
      <td>Code lauffähig und korrekt</td>
      <td>Der Code läuft einwandfrei und liefert das korrekte Ergebnis ohne Mängel.</td>
      <td>100% der max. erreichbaren Punkte</td>
    </tr>
  </tbody>
</table>


#### <b>Python Libraries und Settings</b>

In [5]:
# Libraries
import os
import folium
import pandas as pd
import geopandas as gpd
from IPython.display import display, IFrame
from sqlalchemy import create_engine, text

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

print(os.getcwd())


/workspaces/python_postgresql_postgis/EXAM


## <b>Vorbereitung (Hinweis: dieser Teil wird <u>nicht</u> bewertet)</b>

#### <b>1.) Starten Sie eine GitHub Codespaces Instanz auf Basis Ihres Forks des folgenden GitHub Repositories:</b>

##### GitHub-Repository: https://github.com/mario-gellrich-zhaw/python_postgresql_postgis

##### <span style="color: red;"><b>WICHTIG!!! Verwenden Sie eine GitHub Codespaces Instanz mit ausreichend Arbeitsspeicher (4core, 16GB RAM).</b></span>

<b>Hinweis:</b> 
- Im Unterricht wurden bereits sämtliche Installationen und Einstellungen inkl. der Registrierung des Datenbank Servers auf pgAdmin vorgenommen.
- Falls Sie die Codespaces-Instanz neu erstellen müssen, folgen Sie bitte den detaillierten Erklärungen auf der README-Seite des GitHub Repositories.

#### <b>2.) Erstellen und Testen Sie die Datenbankverbindung mit der 'osm_switzerland' Datenbank.</b>

In [6]:
# Set up Database Connection
user = "pgadmin"
password = "geheim"
host = "localhost"
port = "5432"
database = "osm_switzerland"

# Erstellen der Connection URL
db_connection_url = "postgresql://" + user + ":" + password +\
                    "@" + host + ":" + port + "/" + database

# Erstellen SQLAlchemy Engine
engine = create_engine(db_connection_url)

# Test der Connection
try:
    with engine.connect() as connection:
        result = connection.execute(text("SELECT current_database();"))
        db_name = result.scalar()
        print("Aktuelle Datenbank:", db_name)
except Exception as e:
    print("Fehler bei der Verbindung:", e)

# Verbindung trennen
engine.dispose()

Aktuelle Datenbank: osm_switzerland


#### <b>3.) Legen Sie einen neuen Order 'maps' im Arbeitsverzeichnis an. In diesem werden Ihre generierten Karten abgespeichert.</b>

<b style="color:red;">Hinweis: Der Order 'maps' inklusive die erzeugten Karten muss als Teil des AP03 mit auf EXAM-Moodle abgegeben werden!</b>

In [7]:
# Anlegen eines Ordners 'maps'
if not os.path.exists('maps'):
    os.makedirs('maps')
    print("Ordner 'maps' wurde erstellt.")
else:
    print("Ordner 'maps' existiert bereits und wird nicht neu angelegt!")

Ordner 'maps' wurde erstellt.


## <b>Aufgaben (Dieser Teil wird bewertet!)</b>
<b>Hinweise zu den folgenden Aufgabenstellungen:</b>
<ul>
  <li>In diesem Jupyter Notebook gibt es jeweils zwei Code-Zellen pro Aufgabe:</li>
  <ol>
    <li>Eine Codezelle mit Python-Code für die SQL-Abfrage.</li>
    <li>Eine Codezelle mit Python-Code für die Kartendarstellung der Ergebnisse der jeweiligen SQL-Abfrage.</li>
  </ol>
  <li>In den Codezellen für die SQL-Abfrage muss jeweils die SQL-Statement ergänzt werden.</li>
  <li>In den Codezellen für die Kartendarstellung muss nur dann der Python Code ergänzt werden, wenn in der Aufgabe danach gefragt wird.</li>
</ul>
<b style="color:red;">Beachten Sie, dass für die Punktevergabe auch die weiteren Anforderungen zu den Fragen unter 'Details zur Aufgabenstellung' herangezogen werden.</b>

#### <b>Aufgabe (1): Erstellen Sie eine Abfrage der Spitäler.</b>
<b>Details zur Aufgabenstellung:</b>
- Sie finden die benötigten Daten in der Tabelle 'planet_osm_polygon'.
- Stellen sie in der Ergebnistabelle die Spalten: osm_id, amenity sowie die transformierte Geometrie als Spalte geom dar.
- Tipps:
    - Die Geometry wird mit Hilfe der Funktion st_transform() transformiert, z.B.: *st_transform(p.way, 4326) AS geom*.
    - Die Daten sind mit dem key:value Paar amenity='hospital' in der OpenStreetMap Map-Feature Übersicht angegeben.
    - vgl: https://wiki.openstreetmap.org/wiki/Map_features

<b style="color: gray;">(max. erreichbare Punkte: 4)</b>

In [9]:
# Engine für Datenbankverbindung erstellen
engine = create_engine(db_connection_url)

# Ergänzen Sie die SQL-Abfrage, um die Aufgabe zu lösen
sql = """SELECT
            p.osm_id,
            p.amenity,
            st_transform(p.way, 4326) AS geom
        FROM
            planet_osm_polygon AS p
        WHERE p.amenity = 'hospital';"""

# Ergebnis in GeoDataFrame abspeichern
gdf_hospital = gpd.GeoDataFrame.from_postgis(sql, engine)

# Datenbankverbindung trennen
engine.dispose()

# Zeigen des GeoDataFrames
gdf_hospital.head(10)

,osm_id,amenity,geom
0,856698475,hospital,"POLYGON ((8.56529 47.44926, 8.56576 47.44914, ..."
1,558245721,hospital,"POLYGON ((8.55653 47.38088, 8.55655 47.3808, 8..."
2,-18241258,hospital,"POLYGON ((8.57208 47.35379, 8.57228 47.35284, ..."
3,-18241258,hospital,"POLYGON ((8.56919 47.3518, 8.56975 47.35114, 8..."
4,527550642,hospital,"POLYGON ((8.72283 47.35273, 8.72304 47.35269, ..."
5,-9065183,hospital,"POLYGON ((8.45701 47.28345, 8.4573 47.28318, 8..."
6,-9065183,hospital,"POLYGON ((8.45507 47.28548, 8.45516 47.28523, ..."
7,42156983,hospital,"POLYGON ((8.5172 47.41482, 8.51749 47.41474, 8..."
8,672118527,hospital,"POLYGON ((8.42825 47.39402, 8.42827 47.39399, ..."
9,673609234,hospital,"POLYGON ((8.44987 47.47785, 8.44994 47.47757, ..."


##### <b>Kartendarstellung Ergebnis (nur anpassen, falls in der Aufgabe danach gefragt wird)</b>

In [10]:
# Projektion definieren (WGS84)
if gdf_hospital.crs is None:
    gdf_hospital.set_crs(epsg=4326, inplace=True)
else:
    pass

# Latitude und Longitude für die Zentrierung der Karte ermitteln
centroids = gdf_hospital.geometry.centroid
lon = centroids.x.mean()
lat = centroids.y.mean()

# Initialisieren der Map
m = folium.Map(location=[lat, lon], 
               zoom_start=11, 
               tiles='CartoDBPositron')

# Map settings
folium.GeoJson(
    gdf_hospital,
    name='map'
).add_to(m)

folium.LayerControl().add_to(m)

# Save the map to an HTML file
m.save('maps/map_hospital.html')
print("Die Karte wurde unter 'maps/map_hospital.html' gespeichert.")

Die Karte wurde unter 'maps/map_hospital.html' gespeichert.


#### <b>Aufgabe (2) Erstellen Sie eine Abfrage der SBB Bahnhöfe.</b>
<b>Details zur Aufgabenstellung:</b>
- Verwenden Sie die Tabelle 'planet_osm_point'.
- Schränken Sie nach Bahnhöfen der SBB ein.
- Stellen Sie in der Ergebnistabelle die Spalten: osm_id, name, operator, public_transport und die transformierte Geometrie als Spalte geom dar.
- Tipp: Die Daten sind mit dem key:value Paar public_transport='station' in der OpenStreetMap Map-Feature Übersicht angegeben.
-  vgl: https://wiki.openstreetmap.org/wiki/Map_features

<b style="color: gray;">(max. erreichbare Punkte: 4)</b>

In [31]:
# Engine für Datenbankverbindung erstellen
engine = create_engine(db_connection_url)  

# Ergänzen Sie die SQL-Abfrage, um die Aufgabe zu lösen
sql = """SELECT
            p.osm_id,
            p.name,
            p.operator,
            p.public_transport
            ST_TRANSFORM(p.way::geometry, 4326) AS geom
        FROM planet_osm_point AS p
        WHERE
            p.public_transport = 'station';"""

# Ergebnis in GeoDataFrame abspeichern
gdf_stations = gpd.GeoDataFrame.from_postgis(sql, engine)

# Datenbankverbindung trennen
engine.dispose()

# Zeigen des GeoDataFrames
gdf_stations.head(10)

ProgrammingError: (psycopg2.errors.SyntaxError) syntax error at or near "("
LINE 6:             ST_TRANSFORM(p.way::geometry, 4326) AS geom
                                ^

[SQL: SELECT
            p.osm_id,
            p.name,
            p.operator,
            p.public_transport
            ST_TRANSFORM(p.way::geometry, 4326) AS geom
        FROM planet_osm_point AS p
        WHERE
            p.public_transport = 'station';]
(Background on this error at: https://sqlalche.me/e/20/f405)

##### <b>Kartendarstellung Ergebnis (nur anpassen, falls in der Aufgabe danach gefragt wird)</b>

In [ ]:
# Projektion definieren (WGS84)
if gdf_stations.crs is None:
    gdf_stations.set_crs(epsg=4326, inplace=True)
else:
    pass

# Latitude und Longitude für die Zentrierung der Karte ermitteln
centroids = gdf_stations.geometry.centroid
lon = centroids.x.mean()
lat = centroids.y.mean()

# Initialisieren der Map
m = folium.Map(location=[lat, lon], 
               zoom_start=10, 
               tiles='CartoDBPositron')

# Map settings with popup
folium.GeoJson(
    gdf_stations,
    name='map'
).add_to(m)

folium.LayerControl().add_to(m)

# Save the map to an HTML file
m.save('maps/map_stations.html')
print("Die Karte wurde unter 'maps/map_stations.html' gespeichert.")

#### <b>Aufgabe (3): Erstellen Sie eine Abfrage aller Strassen, die als 'primary', 'secondary' oder 'tertiary' klassifiziert sind mit Infos zur Art des Fahrradwegs.</b>
<b>Details zur Aufgabenstellung:</b>
- Sie finden die benötigten Informationen in der Tabelle 'planet_osm_line'.
- Stellen Sie in der Ergebnistabelle die Spalten: osm_id, highway, bicycle und die transformierte Geometrie als Spalte geom dar.
- Schränken Sie die Abfrage auf alle Strassen ein, bei denen Information zur Art des Fahrradweges vorliegen.
- Tipp: Sie finden die Informationen zur Art der Fahrradwegs in der Spalte 'bicycle'.
-  vgl: https://wiki.openstreetmap.org/wiki/Map_features

<b style="color: gray;">(max. erreichbare Punkte: 4)</b>

In [21]:
# Engine für Datenbankverbindung erstellen
engine = create_engine(db_connection_url)  

# Ergänzen Sie die SQL-Abfrage, um die Aufgabe zu lösen
sql = """SELECT
            p.osm_id,
            p.highway,
            p.bicycle,
            st_transform(p.way, 4326) AS geom
        FROM
            planet_osm_polygon AS p
        WHERE
            p.street IS NOT NULL;"""

# Ergebnis in GeoDataFrame abspeichern
gdf_streets = gpd.GeoDataFrame.from_postgis(sql, engine)

# Datenbankverbindung trennen


# Zeigen des GeoDataFrames
gdf_streets.head(10)

ProgrammingError: (psycopg2.errors.UndefinedColumn) column p.street does not exist
LINE 9:             p.street IS NOT NULL;
                    ^

[SQL: SELECT
            p.osm_id,
            p.highway,
            p.bicycle,
            st_transform(p.way, 4326) AS geom
        FROM
            planet_osm_polygon AS p
        WHERE
            p.street IS NOT NULL;]
(Background on this error at: https://sqlalche.me/e/20/f405)

##### <b>Kartendarstellung Ergebnis (nur anpassen, falls in der Aufgabe danach gefragt wird)</b>

In [ ]:
# Projektion definieren (WGS84)
if gdf_streets.crs is None:
    gdf_streets.set_crs(epsg=4326, inplace=True)
else:
    pass

# Latitude und Longitude für die Zentrierung der Karte ermitteln
centroids = gdf_streets.geometry.centroid
lon = centroids.x.mean()
lat = centroids.y.mean()

# Initialisieren der Map
m = folium.Map(location=[lat, lon], 
               zoom_start=11, 
               tiles='CartoDBPositron')

# Map settings
folium.Choropleth(
    geo_data=gdf_streets,
    name='map',
    line_weight=3,
    line_color='darkred'
).add_to(m)

folium.LayerControl().add_to(m)

# Save the map to an HTML file
m.save('maps/map_streets.html')
print("Die Karte wurde unter 'maps/map_streets.html' gespeichert.")

#### <b>Aufgabe (4): Erstellen Sie eine Abfrage aller Gebäude in der Bahnhofstrasse in Zürich, bei denen mindestens die Postleitzahl (PLZ) vorhanden ist.</b>
<b>Details zur Aufgabenstellung:</b>
- Verwenden Sie die Tabelle 'planet_osm_polygon'.
- Stellen Sie in der Ergebnistabelle Strassenname, Haunummer, PLZ, Gemeindename sowie die transformierte Geometrie als Spalte geom dar.
- Dabei ist nicht wichtig, dass alle Adress-Attribute für jedes Gebäude vorhanden sind, nur die PLZ muss immer vorhanden sein.
- Verwenden Sie für die Darstellung als Hintergrundkarte ein Satellitenbild (ESRIWorldImagery) als maptile.
- Tipp: Gebäude sind in der Spalte 'building' klassifiziert. Mit *WHERE building IS NOT NULL* können Sie Gebäude filtern.

<b style="color: gray;">(max. erreichbare Punkte: 4)</b>

In [22]:
# Engine für Datenbankverbindung erstellen
engine = create_engine(db_connection_url)  

# Ergänzen Sie die SQL-Abfrage, um die Aufgabe zu lösen
sql = """SELECT 
            p.Strassenname,
            p.Hausnummer,
            p.PLZ
            ST_TRANSFORM(p.way::geometry, 4326) AS geom
        FROM planet_osm_polygon AS p
        WHERE p.buildings = 'motorway';"""


# Ergebnis in GeoDataFrame abspeichern
gdf_buildings = gpd.GeoDataFrame.from_postgis(sql, engine)

# Datenbankverbindung trennen


# Zeigen des GeoDataFrames
gdf_buildings.head(10)

ProgrammingError: (psycopg2.errors.SyntaxError) syntax error at or near "("
LINE 5:             ST_TRANSFORM(p.way::geometry, 4326) AS geom
                                ^

[SQL: SELECT 
            p.Strassenname,
            p.Hausnummer,
            p.PLZ
            ST_TRANSFORM(p.way::geometry, 4326) AS geom
        FROM planet_osm_polygon AS p
        WHERE p.buildings = 'motorway';]
(Background on this error at: https://sqlalche.me/e/20/f405)

##### <b>Kartendarstellung Ergebnis (nur anpassen, falls in der Aufgabe danach gefragt wird)</b>

In [ ]:
# Projektion definieren (WGS84)
if gdf_buildings.crs is None:
    gdf_buildings.set_crs(epsg=4326, inplace=True)
else:
    pass

# Latitude und Longitude für die Zentrierung der Karte ermitteln
centroids = gdf_buildings.geometry.centroid
lon = centroids.x.mean()
lat = centroids.y.mean()

# Initialisieren der Map
m = folium.Map(location=[lat, lon], 
               zoom_start=16, 
               tiles='CartoDBPositron')

# Map settings
folium.Choropleth(
    geo_data=gdf_buildings,
    name='map',
    fill_color='orange'
).add_to(m)

folium.LayerControl().add_to(m)

# Save the map to an HTML file
m.save('maps/map_buildings.html')
print("Die Karte wurde unter 'maps/map_buildings.html' gespeichert.")

#### <b>Aufgabe (5): Erstellen Sie eine Abfrage des Greifensees, und generieren Sie einen Buffer um diesen See mit einer Breite von 500m.</b>
<b>Details zur Aufgabenstellung:</b>
- Sie finden die benötigten Informationen in der Tabelle 'planet_osm_polygon'.
- Stellen Sie in der Ergebnistabelle die Spalten: osm_id, name, natural sowie die transformierte Geometrie des Buffers als Spalte geom dar.
- Im Polygondatensatz gibt es unter dem Namen 'Greifensee' nicht nur einen Polygon. Filtern Sie in der Abfrage nur den See.
- Tipp: Zum Filtern des Sees können Sie die Spalte 'natural' verwenden.
-  vgl: https://wiki.openstreetmap.org/wiki/Map_features

<b style="color: gray;">(max. erreichbare Punkte: 8)</b>

In [23]:
# Engine für Datenbankverbindung erstellen
engine = create_engine(db_connection_url)  

# Ergänzen Sie die SQL-Abfrage, um die Aufgabe zu lösen
sql = """SELECT
            p.osm_id,
            p.name,
            p.natural
            ST_Transform(ST_Buffer(ST_Transform(p.way::geometry, 3857), 2000), 4326) AS geom
        FROM
            planet_osm_polygon AS p
        WHERE
            p.natural = 'lake';"""

# Ergebnis in GeoDataFrame abspeichern
gdf_lake = gpd.GeoDataFrame.from_postgis(sql, engine)

# Datenbankverbindung trennen


# Zeigen des GeoDataFrames
gdf_lake

ProgrammingError: (psycopg2.errors.SyntaxError) syntax error at or near "("
LINE 5:             ST_Transform(ST_Buffer(ST_Transform(p.way::geome...
                                ^

[SQL: SELECT
            p.osm_id,
            p.name,
            p.natural
            ST_Transform(ST_Buffer(ST_Transform(p.way::geometry, 3857), 2000), 4326) AS geom
        FROM
            planet_osm_polygon AS p
        WHERE
            p.natural = 'lake';]
(Background on this error at: https://sqlalche.me/e/20/f405)

##### <b>Kartendarstellung Ergebnis (nur anpassen, falls in der Aufgabe danach gefragt wird)</b>

In [ ]:
# Projektion definieren (WGS84)
if gdf_lake.crs is None:
    gdf_lake.set_crs(epsg=4326, inplace=True)
else:
    pass

# Latitude und Longitude für die Zentrierung der Karte ermitteln
centroids = gdf_lake.geometry.centroid
lon = centroids.x.mean()
lat = centroids.y.mean()

# Initialisieren der Map
m = folium.Map(location=[lat, lon], 
               zoom_start=13, 
               tiles='ESRIWorldImagery')

# Map settings
folium.Choropleth(
    geo_data=gdf_lake,
    name='map',
    fill_color='greenyellow',
    fill_opacity=0.5,
    line_opacity=0.5
).add_to(m)

folium.LayerControl().add_to(m)

# Save the map to an HTML file
m.save('maps/map_lake.html')
print("Die Karte wurde unter 'maps/map_lake.html' gespeichert.")

#### <b>Aufgabe (6): Erstellen Sie eine Abfrage der landwirtschaftlichen Nutzflächen in den Grenzen der Stadt Zürich.</b>
<b>Details zur Aufgabenstellung:</b>
- Sie finden die benötigten Daten in der Tabelle 'planet_osm_polygon'.
- Landwirtschaftliche Nutzflächen sollen in dieser Aufgabe sowohl 'farmland' als auch 'meadows' beinhalten.
- Selektieren Sie die landwirtschaftlichen Nutzflächen über die Gemeindefläche.
- Berechen Sie die Fläche der landwirtschaftlichen Nutzflächen und speichern diese in einer Spalte 'area_ha' ab.
- Stellen Sie in der Ergebnistabelle die Spalten: osm_id, landuse, area_ha sowie die transformierte Geometrie als geom dar.
- Wählen Sie eine Satelliten Karte von ESRI als Hintergrundkarte (maptile).
- Ordnen Sie die Ergebnistabelle nach area_ha absteigend.
- Tipp: Im Unterricht wurde eine ähnliche Abfrage mit Waldflächen (forest) behandelt.
- vgl: https://wiki.openstreetmap.org/wiki/Map_features

<b style="color: gray;">(max. erreichbare Punkte: 8)</b>

In [ ]:
# Engine für Datenbankverbindung erstellen
engine = create_engine(db_connection_url)  

# Ergänzen Sie die SQL-Abfrage, um die Aufgabe zu lösen
sql = """SELECT
         FROM
         WHERE;"""

# Ergebnis in GeoDataFrame abspeichern
gdf_agriculture = gpd.GeoDataFrame.from_postgis(sql, engine)

# Datenbankverbindung trennen


# Zeigen des GeoDataFrames
gdf_agriculture.head(10)

##### <b>Kartendarstellung Ergebnis (nur anpassen, falls in der Aufgabe danach gefragt wird)</b>

In [ ]:
# Projektion definieren (WGS84)
if gdf_agriculture.crs is None:
    gdf_agriculture.set_crs(epsg=4326, inplace=True)
else:
    pass

# Latitude und Longitude für die Zentrierung der Karte ermitteln
centroids = gdf_agriculture.geometry.centroid
lon = centroids.x.mean()
lat = centroids.y.mean()

# Initialisieren der Map
m = folium.Map(location=[lat, lon], 
               zoom_start=13, 
               tiles='EsriWorldImagery')

# Map settings
folium.GeoJson(
    gdf_agriculture,
    name='map',
).add_to(m)

folium.LayerControl().add_to(m)

# Save the map to an HTML file
m.save('maps/map_farmland_meadow.html')
print("Die Karte wurde unter 'maps/map_farmland_meadow.html' gespeichert.")

#### <b>Aufgabe (7): Erstellen Sie eine Abfrage der Hotels in einem Radius von 5km um den Flughafen Zürich.</b>
<b>Details zur Aufgabenstellung:</b>
- Verwenden Sie die Tabelle 'planet_osm_point'.
- Berechnen Sie in der Abfrage die Distanz jedes Hotels zum Flughafen in Metern als Spalte 'distance_meters'.
- Stellen Sie in der Ergebnistabelle die Spalten: osm_id, tourism, name, distance_meters, sowie die transformierte Geometrie als geom dar.
- Wählen Sie eine Satelliten Karte von ESRI als Hintergrundkarte (maptile).
- Integrieren Sie in die Kartendarstellung den Namen (Spalte 'name') der Hotels als Popup.
- Tipp: Hotels sind mit dem key:value Paar tourism='hotel' in der OpenStreetMap Map-Feature Übersicht angegeben.
- vgl: https://wiki.openstreetmap.org/wiki/Map_features

<b style="color: gray;">(max. erreichbare Punkte: 8)</b>

In [30]:
# Engine für Datenbankverbindung erstellen
engine = create_engine(db_connection_url)  

# Ergänzen Sie die SQL-Abfrage, um die Aufgabe zu lösen
sql = """SELECT
            p.osm_id,
            p.tourism,
            p.name,
            ST_Distance(
                ST_Transform(p.way, 4326)::geography,
                ST_SetSRID(ST_MakePoint(8.56322, 47.45424), 4326)::geography
            ) AS distance_meters,
            ST_TRANSFORM(p.way, 4326) AS geom
        FROM
            planet_osm_point AS p
        WHERE
            p.shop = 'hotel'
            AND ST_DWithin(
                ST_Transform(p.way, 4326)::geography,
                ST_SetSRID(ST_MakePoint(8.56322, 47.45424), 4326)::geography,
                5000
            )
        ORDER BY distance_meters;"""


# Ergebnis in GeoDataFrame abspeichern
gdf_hotels = gpd.GeoDataFrame.from_postgis(sql, engine)

# Datenbankverbindung trennen
engine.dispose()


# Zeigen des GeoDataFrames
gdf_hotels.head(10)


,osm_id,tourism,name,distance_meters,geom


In [28]:
# Projektion definieren (WGS84)
if gdf_hotels.crs is None:
    gdf_hotels.set_crs(epsg=4326, inplace=True)
else:
    pass

# Latitude und Longitude für die Zentrierung der Karte ermitteln
centroids = gdf_hotels.geometry.centroid
lon = centroids.x.mean()
lat = centroids.y.mean()

# Initialisieren der Map
m = folium.Map(location=[lat, lon], 
               zoom_start=13, 
               tiles='EsriWorldImagery')

# Map settings
folium.GeoJson(
    gdf_hotels,
    name='map',
    popup=folium.GeoJsonPopup(fields=['name'])
).add_to(m)

folium.LayerControl().add_to(m)

# Save the map to an HTML file
m.save('maps/map_hotels.html')
print("Die Karte wurde unter 'maps/map_hotels.html' gespeichert.")

ValueError: Location values cannot contain NaNs.

### Jupyter notebook --footer info-- (please always provide this at the end of each notebook)

In [ ]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('IP Address:', socket.gethostbyname(socket.gethostname()))
print('-----------------------------------')